# 2019 Novel Coronavirus (2019-nCoV)

Coronaviruses (CoV) are a large family of viruses that cause illness ranging from the common cold to more severe diseases such as Middle East Respiratory Syndrome (MERS-CoV) and Severe Acute Respiratory Syndrome (SARS-CoV). A novel coronavirus (nCoV) is a new strain that has not been previously identified in humans.  

Coronaviruses are zoonotic, meaning they are transmitted between animals and people.  Detailed investigations found that SARS-CoV was transmitted from civet cats to humans and MERS-CoV from dromedary camels to humans. Several known coronaviruses are circulating in animals that have not yet infected humans. 


![](https://www.1mg.com/articles/wp-content/uploads/2020/01/coronavirus.jpg)

> 2019 Novel Coronavirus (2019-nCoV) is a virus (more specifically, a coronavirus) identified as the cause of an outbreak of respiratory illness first detected in Wuhan, China. Early on, many of the patients in the outbreak in Wuhan, China reportedly had some link to a large seafood and animal market, suggesting animal-to-person spread. However, a growing number of patients reportedly have not had exposure to animal markets, indicating person-to-person spread is occurring. At this time, it’s unclear how easily or sustainably this virus is spreading between people.


*  [How it Spreads](https://www.cdc.gov/coronavirus/2019-ncov/about/transmission.html)
* [Symptoms](https://www.cdc.gov/coronavirus/2019-ncov/about/symptoms.html)
*  [Prevention & Treatment](https://www.cdc.gov/coronavirus/2019-ncov/about/prevention-treatment.html)

The **World Health Organization** (WHO) has announce that "**COVID-19**" will now be the official name of the deadly virus from
China, saying the disease represented a "very grave threat" for the world but there was a "realistic chance" of stopping it.
WHO renames deadly coronavirus to avoid references to specific location, animal species or group of people. [Reference](https://www.aljazeera.com/news/2020/02/covid-19-renames-deadly-coronavirus-200211172638418.html?utm_source=website&utm_medium=article_page&utm_campaign=read_more_links)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio


from datetime import date


sns.set(style='whitegrid')
pio.templates.default = "plotly_white"
%matplotlib inline
warnings.filterwarnings('ignore')


df = pd.read_csv('../input/novel-corona-virus-2019-dataset/covid_19_data.csv')
world_coor = pd.read_csv('../input/world-coor/world_coordinates.csv')

df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.columns = df.columns.str.replace(' ','_').str.replace('/','_')
df.rename(columns={'Country_Region':'Country'}, inplace=True)
df['Date'] = df['ObservationDate'].apply(pd.to_datetime)
df['Last_Update'] = pd.to_datetime(df.Last_Update)
df.drop('SNo', axis=1, inplace=True)
df.Country.replace({'Mainland China': 'China'}, inplace=True)
df = df.astype({'Confirmed':'int','Deaths':'int','Recovered':'int'})

d = df['Date'][-1:].astype('str')
year = int(d.values[0].split('-')[0])
month = int(d.values[0].split('-')[1])
day = int(d.values[0].split('-')[2].split()[0])

df_1 = df[df['Date'] >= pd.Timestamp(date(year,month,day))]
df_1

In [ ]:
df['Date'] = df['Date'].dt.date
glob_spread = df[df['Date'] > pd.Timestamp(date(2020,1,21))]
glob_spread = glob_spread.groupby('Date')["Confirmed", "Deaths", "Recovered"].sum().reset_index()

total_cases = df_1[['Confirmed','Deaths','Recovered']].sum()
total_cases = total_cases.to_frame().reset_index()
total_cases.rename(columns={'index':'types',0:'Total'}, inplace=True)
labels = total_cases.types
values = total_cases.Total

colors = ['STEELBLUE','crimson','MEDIUMSEAGREEN']

fig_1 = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='label+percent', showlegend=False)])

fig_1.update_traces(marker=dict(colors=colors, line=dict(color='#000000', width=0.5)))

fig_1.update_layout( margin={"r":0,"t":100,"l":0,"b":0},
    title_text="<br>COVID-19 Global Cases<br>",
    font=dict(size=15, color='black', family="Arial, Balto, Courier New, Droid Sans"),

)
fig_1.show()

fig_2 = go.Figure()
fig_2.add_trace(go.Scatter(x=glob_spread['Date'], y=glob_spread['Confirmed'], name='Confirmed',
                         line=dict(color='STEELBLUE', width=4),  mode='lines+markers'))

fig_2.add_trace(go.Scatter(x=glob_spread['Date'], y=glob_spread['Recovered'], name = 'Recovered',
                         line=dict(color='MEDIUMSEAGREEN', width=4),mode='lines+markers'))
                
fig_2.add_trace(go.Scatter(x=glob_spread['Date'], y=glob_spread['Deaths'], name='Deaths',
                         line=dict(color='crimson', width=4),mode='lines+markers'
                          ))
                
fig_2.update_layout(yaxis_title='COVID-19 Total Cases',
    xaxis=dict(
        showline=False,
        showgrid=False,
        showticklabels=True,
        linecolor='black',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    #plot_bgcolor='white'
)

fig_2.show()

# [Number of Countries affected by Coronavirus Disease 19](http://)

In [ ]:
count = df.Country.unique()
cases = df_1.groupby('Country')[['Confirmed','Deaths','Recovered']].sum()
most_count = cases.sort_values(by=['Confirmed','Deaths','Recovered'], ascending=False)
df_world = pd.merge(world_coor,cases,on='Country')

print(count)
print('-----'*20)
print('Number of Countries affected by NCov Virus: ',len(count))

fig_3 = go.Figure()
fig_3.add_trace(go.Scattergeo(
        lon = df_world['longitude'],
        lat = df_world['latitude'],
        mode='markers',
         marker=dict(
            size=10,
            color='rgb(255, 0, 0)',
            opacity=0.7
         ),
        text = df_world[['Country','Confirmed']],
        hoverinfo='text'))

fig_3.update_layout(mapbox_style="carto-darkmatter",
        margin={"r":0,"t":100,"l":0,"b":0},
       title=dict(text="<br>Countries with confirmed cases of COVID-19, 2020<br>"),
                             font=dict(size=15, color='black', family="Arial, Balto, Courier New, Droid Sans"),
        geo = dict(
            landcolor = 'rgb(217, 217, 217)',
            #projection_type="equirectangular",
            coastlinecolor = "black",
             bgcolor = "#9cd3db",
            showframe=False,
            showcountries=True,
        
        
            
            
        ),
    )

fig_3.show()

# [The Most Affected Countries by COVID-19](http://)




In [ ]:

plt.figure(figsize=(10,5))
ax = sns.heatmap(most_count[:10], annot=True, fmt="d", linewidths=0.1, cmap='Reds', linecolor='white', cbar=False)

# [Confirmed Cases of COVID-19 in Asia, 2020](http://)




In [ ]:
fig_4 = go.Figure(data=go.Choropleth(
    locations = df_world['Country'],
    z = df_world['Confirmed'],
    locationmode="country names",
    text = df_world['Country'],
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_title = 'Confirmed Cases<br>of COVID-19',
    colorscale = 'peach',
))

fig_4.update_layout(margin={"r":0,"t":40,"l":0,"b":0},
               
    geo=dict(
        showframe=False,
        showcoastlines=True,
        showcountries=True,
        projection_type='equirectangular',
         coastlinecolor = "black",
            countrycolor='black',
        scope='asia'
        
    ),

)

fig_4.show()
#burg, geyser, hsv

# [Top 10 States with Confirmed Cases of COVID-19 in China](http://)

![](https://kitv.images.worldnow.com/images/19140197_G.jpeg)

> The first infected people came from Wuhan City, Hubei Province of China, a city of more than 11 million people and a major transportation hub. Health experts agree that many patients 
> had some association with a large live animal/seafood in market in Wuhan City, suggesting the disease was first spread from animal-to-human contact. Human-to-human transfer was soon confirmed
> in China, Germany, and the United States.

In [ ]:
china_prov = df_1[df_1.Country =='China']
china_prov = china_prov.groupby('Province_State')[['Confirmed','Recovered']].sum().sort_values(by='Confirmed', ascending=False).reset_index()
china_prov = china_prov[:10].sort_values(by='Confirmed', ascending=True)

fig_5 = go.Figure()
fig_5.add_trace(go.Bar(
    y=china_prov['Province_State'],
    x=china_prov['Recovered'],
    text=china_prov['Recovered'],
    textposition='auto',
    name='Recovered',
    orientation='h',
    marker=dict(
        color='STEELBLUE')
    )
)
fig_5.add_trace(go.Bar(
    y=china_prov['Province_State'],
    x=china_prov['Confirmed'],
    text=china_prov['Confirmed'],
    textposition='auto',
    name='Confirmed',
    orientation='h',
    marker=dict(
        color='crimson',
    )
    )
)

fig_5.update_layout(
    yaxis=dict(
        title='States',
        titlefont_size=16,
        tickfont_size=14),
    barmode='group',
    margin=dict(l=0, r=0, t=20, b=0))
    
fig_5.show()

# [Europe is now the ‘epicenter’ of the coronavirus pandemic](http://)
![](https://www.barkwithassociates.com/website/wp-content/uploads/2019/06/EU-1.jpg)

> Europe has become the new epicenter of the COVID-19 pandemic as cases in China slow and the deadly coronavirus runs through Italy and nearby countries, World Health Organization officials said.

> More cases are now being reported in Europe  every day than were reported in China at the height of its epidemic,” WHO Director-General Dr. Tedros Adhanom Ghebreyesus said at a news conference at the organization’s Geneva headquarters.

> WHO officials declared COVID-19 a global pandemic  as the virus spreads rapidly across the world from Asia to Europe, the Middle East and now parts of the United States.

>  Italy currently has the most cases outside of China with at least 15,113 infections, followed by Spain at 4,334, Germany at 3,156 and France at 2,882, according to data compiled by Johns Hopkins University. Other countries in Europe are seeing cases soar. Switzerland currently has 1,125 cases, followed by Sweden at 809, the Netherlands at 804 and Denmark at 788. - March 13,2020

In [ ]:
fig_6 = go.Figure(data=go.Choropleth(
    locations = df_world['Country'],
    z = df_world['Confirmed'],
    locationmode="country names",
    text = df_world['Country'],
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_title = 'Confirmed Cases<br>of COVID-19',
    colorscale = 'Reds',
))

fig_6.update_layout(margin={"r":0,"t":40,"l":0,"b":0},
               
    geo=dict(
        showframe=False,
        showcoastlines=True,
        showcountries=True,
        projection_type='equirectangular',
         coastlinecolor = "black",
            countrycolor='black',
        scope='europe'
        
    ),

)

fig_6.show()
#burg, geyser, hsv

# [WHO says a third of the newest coronavirus cases worldwide are in the US](http://)

![](https://assetsds.cdnedge.bluemix.net/sites/default/files/styles/big_2/public/feature/images/coronavirus_17.jpg?itok=71rffB4a)

> The coronavirus outbreak reached a somber milestone in the U.S. on Thursday (March 26) — the nation now has more coronavirus cases than any other country in the world. There are now at least 82,174 cases of the new coronavirus disease, COVID-19, in the United States, according to worldometer, a website that tracks coronavirus cases. That's higher than the case count in either Italy (which has  80,589 cases) or China (which has 81,285 cases). Nearly half of all U.S. coronavirus cases are in New York, which has reported more than 37,700 cases.


In [ ]:
fig_8 = go.Figure(data=go.Choropleth(
    locations = df_world['Country'],
    z = df_world['Confirmed'],
    locationmode="country names",
    text = df_world['Country'],
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_title = 'Confirmed Cases<br>of COVID-19',
    colorscale = 'Reds',
))

fig_8.update_layout(margin={"r":0,"t":40,"l":0,"b":0},
               
    geo=dict(
        showframe=False,
        showcoastlines=True,
        showcountries=True,
        projection_type='equirectangular',
         coastlinecolor = "black",
            countrycolor='black',
        scope='usa'
        
    ),

)

fig_8.show()
#burg, geyser, hsv

# [Top 10 States with Confirmed Cases of COVID-19 in US](http://)

In [ ]:
us_prov = df_1[df_1.Country =='US']
us_prov = us_prov.groupby('Province_State')[['Confirmed','Recovered']].sum().sort_values(by='Confirmed', ascending=False).reset_index()
us_prov = us_prov[:10].sort_values(by='Confirmed', ascending=True)

fig_9 = go.Figure()
fig_9.add_trace(go.Bar(
    y=us_prov['Province_State'],
    x=us_prov['Recovered'],
    text=us_prov['Recovered'],
    textposition='auto',
    name='Recovered',
    orientation='h',
    marker=dict(
        color='STEELBLUE')
    )
)
fig_9.add_trace(go.Bar(
    y=us_prov['Province_State'],
    x=us_prov['Confirmed'],
    text=china_prov['Confirmed'],
    textposition='auto',
    name='Confirmed',
    orientation='h',
    marker=dict(
        color='crimson',
    )
    )
)

fig_9.update_layout(
    yaxis=dict(
        title='States',
        titlefont_size=16,
        tickfont_size=14),
    barmode='group',
    margin=dict(l=0, r=0, t=20, b=0))
    
fig_9.show()

# Reference:
* https://www.kaggle.com/vikassingh1996/coronavirus-an-exploratory-study-w-detail-report
* https://www.onhealth.com/content/1/2019-ncov_wuhan_coronavirus_outbreak
* https://time.com/5782284/who-name-coronavirus-covid-19/
* https://www.statnews.com/2020/02/11/disease-caused-by-the-novel-coronavirus-has-name-covid-19/
* https://www.who.int/health-topics/coronavirus
* https://www.1mg.com/articles/wp-content/uploads/2020/01/coronavirus.jpg
* https://kitv.images.worldnow.com/images/19140197_G.jpeg
* https://www.cdc.gov/coronavirus/2019-ncov/
* https://www.aljazeera.com/news/2020/02/covid-19-renames-deadly-coronavirus-200211172638418.html?utm_source=website&utm_medium=article_page&utm_campaign=read_more_links
* https://www.cnbc.com/2020/03/13/europe-is-now-the-epicenter-of-the-coronavirus-pandemic-who-says.html
* http://www.cidrap.umn.edu/news-perspective/2020/03/who-europe-now-worlds-covid-19-epicenter
* https://www.livescience.com/us-coronavirus-cases-surpass-china.html
* https://www.vox.com/2020/3/24/21192352/new-coronavirus-cases-worldwide-united-states-who
* https://www.statnews.com/2020/03/21/coronavirus-plea-from-italy-treat-patients-at-home/

